In [3]:
import time
import calendar
import datetime
import matplotlib.pyplot as plt
import ast
import pandas as pd
import sklearn
from sklearn.model_selection import ShuffleSplit , StratifiedKFold , RepeatedStratifiedKFold ,train_test_split
from sklearn.model_selection import cross_val_score , cross_validate ,cross_val_predict
from sklearn.metrics import plot_roc_curve, plot_precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve, precision_recall_curve , classification_report
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
import seaborn as sns
import warnings
import itertools
warnings.filterwarnings('ignore')
plt.style.use('ggplot')
sns.set_palette(sns.husl_palette(9, s=0.7 ))
sns.set_style(style='white')
import plotly.express as px
import numpy as np
import nltk
from sklearn.impute import KNNImputer
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
#imports related to modeling
import numpy as np
from gensim.models import Word2Vec, KeyedVectors
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


data = pd.read_csv(r'C:\Users\ABC\Downloads\data_ted_talks.csv')
df = data.copy()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ABC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ABC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4005 entries, 0 to 4004
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   talk_id         4005 non-null   int64  
 1   title           4005 non-null   object 
 2   speaker_1       4005 non-null   object 
 3   all_speakers    4001 non-null   object 
 4   occupations     3483 non-null   object 
 5   about_speakers  3502 non-null   object 
 6   views           4005 non-null   int64  
 7   recorded_date   4004 non-null   object 
 8   published_date  4005 non-null   object 
 9   event           4005 non-null   object 
 10  native_lang     4005 non-null   object 
 11  available_lang  4005 non-null   object 
 12  comments        3350 non-null   float64
 13  duration        4005 non-null   int64  
 14  topics          4005 non-null   object 
 15  related_talks   4005 non-null   object 
 16  url             4005 non-null   object 
 17  description     4005 non-null   o

In [4]:
df['all_speakers'].fillna("{0:['NaN']}",inplace=True,axis=0)
df['all_speakers'] = df['all_speakers'].apply(ast.literal_eval)

df['occupations'].fillna("{0:['NaN']}",inplace=True,axis=0)
df['occupations'] = df['occupations'].apply(ast.literal_eval)

df['about_speakers'].fillna("{0:['NaN']}",inplace=True,axis=0)
df['about_speakers'] = df['about_speakers'].apply(ast.literal_eval)

df['published_date'] = df['published_date'].apply(lambda x: datetime.datetime.strptime((x),"%Y-%m-%d"))

df.drop(df[df['recorded_date'].isna()].index[0], inplace = True)
df['recorded_date'] = df['recorded_date'].apply(lambda x: datetime.datetime.strptime((x),"%Y-%m-%d"))

df['total_days_since_published'] = (df['published_date'].max() - df['published_date']).apply(lambda x : x.days )

month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
day_order   = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

df['month'] = pd.DatetimeIndex(df['published_date']).month
df['month'] = df['month'].apply(lambda x: calendar.month_abbr[x])
df['year'] = pd.DatetimeIndex(df['published_date']).year
df['day'] = pd.DatetimeIndex(df['published_date']).day
df['week_day']= df['published_date'].apply(lambda x: day_order[datetime.date(x.year, x.month, x.day).weekday()])

# Adding a daily views column
df['views_received_daily'] = df['views'] / ( df['total_days_since_published'] + 1 )

df['speaker_1_average_views'] = df['speaker_1'].map(list(df.groupby('speaker_1').agg({'views_received_daily' : 'mean'}).sort_values(['views_received_daily'],ascending=False).to_dict().values())[0])

df['event_average_views'] = df['event'].map(list(df.groupby('event').agg({'views_received_daily' : 'mean'}).sort_values(['views_received_daily'],ascending=False).to_dict().values())[0])

df.reset_index(drop=True, inplace = True)

#Checking for unique topic
unique_topics=[]
for i in range(0,len(df)):
  temp=df['topics'][i]
  for i in temp:
    if(i not in unique_topics):
      unique_topics.append(i)
      
len(unique_topics)

unique_topics_avg_view_dict={}
for topic in unique_topics:
  temp, count = 0, 0

  for i in range(0,len(df)):
    temp2=df['topics'][i]
    if(topic in temp2):
      temp+=df['views_received_daily'][i]
      count+=1
  unique_topics_avg_view_dict[topic]=temp//count

topics_wise_average_views=[]

for i in range(0,len(df)):
  temp=0
  temp_topic=df['topics'][i]
  for element in temp_topic:
    temp+= unique_topics_avg_view_dict[element]
  
  topics_wise_average_views.append(temp//len(temp_topic))


df['topics_wise_avg_views'] = pd.Series(topics_wise_average_views).values

# Removing outliers
df = df[((df['duration'] - df['duration'].mean()) / df['duration'].std()).abs() < 3]


df['unique_topics'] = ((df['topics'].apply(ast.literal_eval)).apply(set)).apply(len)

cleaneddf = df[['comments', 'duration', 'total_days_since_published', 'month', 'year', 'day',
       'week_day', 'speaker_1_average_views',
       'event_average_views', 'unique_topics', 'views_received_daily', 'transcript']]





In [5]:
cleaneddf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3982 entries, 0 to 4003
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   comments                    3327 non-null   float64
 1   duration                    3982 non-null   int64  
 2   total_days_since_published  3982 non-null   int64  
 3   month                       3982 non-null   object 
 4   year                        3982 non-null   int64  
 5   day                         3982 non-null   int64  
 6   week_day                    3982 non-null   object 
 7   speaker_1_average_views     3982 non-null   float64
 8   event_average_views         3982 non-null   float64
 9   unique_topics               3982 non-null   int64  
 10  views_received_daily        3982 non-null   float64
 11  transcript                  3982 non-null   object 
dtypes: float64(4), int64(5), object(3)
memory usage: 404.4+ KB


In [6]:
cleaneddf = cleaneddf[((cleaneddf['comments'] - cleaneddf['comments'].mean()) / cleaneddf['comments'].std()).abs() < 3]
cleaneddf = cleaneddf[((cleaneddf['speaker_1_average_views'] - cleaneddf['speaker_1_average_views'].mean()) / cleaneddf['speaker_1_average_views'].std()).abs() < 3]


In [7]:
pretrain_w2v = r"C:\Users\ABC\Downloads\GoogleNews-vectors-negative300.bin"
w2v_model = KeyedVectors.load_word2vec_format(pretrain_w2v, binary=True)


def preprocess_corpus(texts):
    mystopwords = set(stopwords.words("english"))
    def remove_stops_digits(tokens):
        #Nested function that lowercases, removes stopwords and digits from a list of tokens
        return [token.lower() for token in tokens if token.lower() not in mystopwords and not token.isdigit()
               and token not in punctuation]
    #This return statement below uses the above function to process twitter tokenizer output further. 
    return [remove_stops_digits(word_tokenize(text)) for text in texts]

texts_processed = preprocess_corpus(cleaneddf['transcript'])
def embedding_feats(list_of_lists):
    DIMENSION = 300
    zero_vector = np.zeros(DIMENSION)
    feats = []
    for tokens in list_of_lists:
        feat_for_this =  np.zeros(DIMENSION)
        count_for_this = 0 + 1e-5 # to avoid divide-by-zero 
        for token in tokens:
            if token in w2v_model:
                feat_for_this += w2v_model[token]
                count_for_this +=1
        if(count_for_this!=0):
            feats.append(feat_for_this/count_for_this) 
        else:
            feats.append(zero_vector)
    return feats

train_vectors = embedding_feats(texts_processed)


In [8]:
final = pd.DataFrame(train_vectors).join(cleaneddf.drop('transcript', axis =1))
final = final.drop((final[final['views_received_daily'].isna()]).index)

In [9]:
for i in final.columns:
    if final[i].isna().sum() > 0:
        print(i)

In [10]:
y=final['views_received_daily']
X=final.drop(columns=['views_received_daily'])

X=pd.get_dummies(X)

scaler = StandardScaler()
imputer = KNNImputer()
imputer.fit(scaler.fit_transform(X))
X = imputer.transform(scaler.fit_transform(X))
X = scaler.inverse_transform(X)


In [14]:
pd.DataFrame(X).to_csv("X.csv")
pd.DataFrame(y).to_csv("y.csv")


In [11]:
xgb_model= xgb.XGBRegressor(objective="reg:squarederror")

X_train, X_test, y_train, y_test= train_test_split(X, y,  test_size= 0.20, random_state= 5)

xgb_model.fit(X_train, y_train)

xgb_r2_train=xgb_model.score(X_train, y_train)
xgb_r2_test= xgb_model.score(X_test, y_test)

xgb_r2_train,xgb_r2_test

(0.999988060375447, 0.8657967308092372)

In [26]:
lr = LinearRegression()
lr.fit(X_train, y_train)
rr = Ridge(alpha=0.01) 
# higher the alpha value, more restriction on the coefficients; low alpha > more generalization,
# in this case linear and ridge regression resembles
rr.fit(X_train, y_train)
rr100 = Ridge(alpha=100) #  comparison with alpha value
rr100.fit(X_train, y_train)
train_score=lr.score(X_train, y_train)
test_score=lr.score(X_test, y_test)
Ridge_train_score = rr.score(X_train,y_train)
Ridge_test_score = rr.score(X_test, y_test)
train_score, test_score, Ridge_train_score, Ridge_test_score

(0.7575826194857351, 0.803618691474815, 0.7572741784132224, 0.8102358498672544)